In [1]:
import pandas as pd
import requests
import numpy as np
import boto3
import time
import botocore
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from sqlalchemy.orm import declarative_base, relationship
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, ForeignKey, text

# Collect hotels data
The goal is the collect to 20 best hotels from the 5 cities with the best weather in the next 7 days.
It will be done using Scrapy on the Booking.com website

In [87]:
# Reading only the first cities as the file is already ordered
cities = pd.read_csv('results/cities.csv', nrows=5)
cities.head()

,city_id,city,lat,lon,mean_feels_like,mean_pop
0,0,Cassis,43.214036,5.539632,19,0.0
1,1,Collioure,42.525050,3.083155,19,0.0
2,2,Carcassonne,43.213036,2.349107,19,0.0
3,3,Marseille,43.296174,5.369953,18,0.0
4,4,Aix en Provence,43.529842,5.447474,18,0.0


In [4]:
# Create an HotelItem class
class HotelItem(scrapy.Item):
    name = scrapy.Field()
    url = scrapy.Field()
    description = scrapy.Field()
    address = scrapy.Field()
    city_id = scrapy.Field()
    lat = scrapy.Field()
    lon = scrapy.Field()
    score = scrapy.Field()
    rating = scrapy.Field()

In [5]:
# List of urls the Spider will have the scrap
urls = [(row['city_id'], f'https://www.booking.com/searchresults.fr.html?ss={row["city"]}&order=score') for index, row in cities.iterrows()]

In [6]:
class BookingSpider(scrapy.Spider):
    # Name of your spider
    name = "bookingspider"

    def start_requests(self): 
        for city_id, url in urls :
            yield scrapy.Request(url, meta={'city_id': city_id})


    def parse(self, response):
        # We only use the first 20 hotels which mean we don't need to scrap more than 1 page per city 
        for link in response.css('a[data-testid="title-link"]')[:20]:
            yield scrapy.Request(link.attrib['href'], callback=self.parse_hotel_details, meta={'city_id': response.meta.get('city_id')})
            
              
    def parse_hotel_details(self, response):
        
        # Itinialise the item container
        items = HotelItem()
        
        # Generic details
        items['name'] = response.css('h2.pp-header__title::text').get()
        items['url'] = response.url.split('?')[0]
        items['description'] = ''.join(response.css('div#property_description_content>p::text').getall())
        
        # Location
        items['address'] = response.css('span.hp_address_subtitle::text').get().strip()
        coordinates = response.css('a#hotel_address').attrib['data-atlas-latlng'].split(',')
        items['lat'] = coordinates[0]
        items['lon'] = coordinates[1]
        items['city_id'] = response.meta.get('city_id')
        
        # Score and ratings
        try:
            items['score'] = float(response.css('div[data-testid="review-score-right-component"]>div.b5cd09854e::text').get().replace(',','.'))
        except: 
            items['score'] = None
        stars_rating = len(response.css('span[data-testid="rating-stars"]>span').getall())
        squares_rating = len(response.css('span[data-testid="rating-squares"]>span').getall())
        items['rating'] = max(stars_rating, squares_rating)
        
        # Return
        yield items
        

In [7]:
# Name of the file where the results will be saved
filename = "top20-hotels.json"

# If file already exists, delete it before crawling (because Scrapy will concatenate the last and new results otherwise)
if filename in os.listdir('results/'):
        os.remove('results/' + filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36',
    'LOG_LEVEL': logging.INFO,
    "AUTOTHROTTLE_ENABLED": True,
    "FEEDS": {
        'results/' + filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(BookingSpider)
process.start()

2022-11-09 11:29:51 [scrapy.utils.log] INFO: Scrapy 2.7.1 started (bot: scrapybot)
2022-11-09 11:29:51 [scrapy.utils.log] INFO: Versions: lxml 4.9.1.0, libxml2 2.9.14, cssselect 1.2.0, parsel 1.7.0, w3lib 2.0.1, Twisted 22.10.0, Python 3.7.15 (default, Oct 25 2022, 06:25:35) - [GCC 8.3.0], pyOpenSSL 22.1.0 (OpenSSL 3.0.7 1 Nov 2022), cryptography 38.0.3, Platform Linux-5.10.124-linuxkit-x86_64-with-debian-10.13
2022-11-09 11:29:51 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'}
2022-11-09 11:29:51 [py.warnings] WARNING: /usr/local/lib/python3.7/site-packages/scrapy/utils/request.py:231: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defi

# Store to s3
Save all the results to a S3 bucket

In [8]:
AWS_ACCESS_KEY="YOUR_KEY"
AWS_SECRET_ACCESS_KEY="YOUR_SECRET_KEY"

In [9]:
session = boto3.Session(aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_ACCESS_KEY)
s3 = session.client("s3")

bucket = s3.create_bucket(Bucket="kayak-project-jedha-marion",
                 CreateBucketConfiguration={'LocationConstraint' : 'eu-west-3'})

In [14]:
s3.put_object(
            Body=cities.to_csv(index=False),
            Bucket='kayak-project-jedha-marion',
            Key='cities_weather.csv',
)

{'ResponseMetadata': {'RequestId': '5YFSKX2RKF3Z8MRA',
  'HostId': 'gc9uo3hO5OazxRSuMPJocMldGoAb3gx8Wq6grdF9MW2D7Lz0GwiqZBN9Igz6O7QB93UqM9kYdSWCVjzqJ3eAaw==',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'gc9uo3hO5OazxRSuMPJocMldGoAb3gx8Wq6grdF9MW2D7Lz0GwiqZBN9Igz6O7QB93UqM9kYdSWCVjzqJ3eAaw==',
   'x-amz-request-id': '5YFSKX2RKF3Z8MRA',
   'date': 'Wed, 09 Nov 2022 12:31:29 GMT',
   'etag': '"157a77f6bf2fc67f4380a2e9f64bf7d8"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"157a77f6bf2fc67f4380a2e9f64bf7d8"'}

In [29]:
hotels = pd.read_json('results/top20-hotels.json')

s3.put_object(
            Body=hotels.to_csv(index=False),
            Bucket='kayak-project-jedha-marion',
            Key='top20-hotels.csv',
)

{'ResponseMetadata': {'RequestId': 'GH3HRD3VBXWHSYZE',
  'HostId': 'ZRoTmokz97NIfeAdffmxpNaFOy6cjPhzfOjgMD1wE/UbHUc3Y2RLpOt+CG8RFNdorRJuQH7TxTg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZRoTmokz97NIfeAdffmxpNaFOy6cjPhzfOjgMD1wE/UbHUc3Y2RLpOt+CG8RFNdorRJuQH7TxTg=',
   'x-amz-request-id': 'GH3HRD3VBXWHSYZE',
   'date': 'Wed, 09 Nov 2022 12:42:42 GMT',
   'etag': '"716ef6a0088894c6f432c2fc9ae02269"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"716ef6a0088894c6f432c2fc9ae02269"'}

# Load to RDS
Get the data from the S3 bucket and add them to a Postgresql database using Sqlachemy

In [44]:
# Load the weather data
s3_client = session.client('s3')
response = s3_client.get_object(Bucket='kayak-project-jedha-marion', Key='cities_weather.csv')
cities_weather = pd.read_csv(io.BytesIO(response.get('Body').read()))

In [45]:
# Load the hotels
response = s3_client.get_object(Bucket='kayak-project-jedha-marion', Key='top20-hotels.csv')
hotels = pd.read_csv(io.BytesIO(response.get('Body').read()))
hotels['hotel_id'] = hotels.index

In [55]:
# Create the database connection
USER_NAME = 'YOUR_USER_NAME'
PASSWORD = 'YOUR_PASSWORD'
HOSTNAME = 'YOUR_HOSTNAME'
DB_NAME = 'YOUR_DB_NAME'

engine = create_engine(f'postgresql+psycopg2://{USER_NAME}:{PASSWORD}@{HOSTNAME}/{DB_NAME}', echo=True)

In [80]:
# Create the cities_weather table
Base = declarative_base()

class CitiesWeather(Base):
    __tablename__ = "cities_weather"

    city_id = Column(Integer, primary_key=True)
    city = Column(String)
    lat = Column(Float)
    lon = Column(Float)
    mean_feels_like = Column(Integer)
    mean_pop = Column(Float)
    hotels = relationship('Hotels', back_populates='city')

Base.metadata.create_all(engine)


2022-11-09 13:33:01,843 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2022-11-09 13:33:01 [sqlalchemy.engine.Engine] INFO: BEGIN (implicit)


2022-11-09 13:33:01,854 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-09 13:33:01 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-09 13:33:01,860 INFO sqlalchemy.engine.Engine [cached since 1439s ago] {'name': 'cities_weather'}


...

2022-11-09 13:33:01 [sqlalchemy.engine.Engine] INFO: 
CREATE TABLE cities_weather (
	city_id SERIAL NOT NULL, 
	city VARCHAR, 
	lat FLOAT, 
	lon FLOAT, 
	mean_feels_like INTEGER, 
	mean_pop FLOAT, 
	PRIMARY KEY (city_id)
)




2022-11-09 13:33:01,911 INFO sqlalchemy.engine.Engine [no key 0.00775s] {}


2022-11-09 13:33:01 [sqlalchemy.engine.Engine] INFO: [no key 0.00775s] {}


2022-11-09 13:33:01,936 INFO sqlalchemy.engine.Engine COMMIT


2022-11-09 13:33:01 [sqlalchemy.engine.Engine] INFO: COMMIT


In [81]:
# Create the hotels table
class Hotels(Base):
    __tablename__ = "hotels"

    hotel_id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String)
    url = Column(String)
    description = Column(String)
    address = Column(String)
    lat = Column(Float)
    lon = Column(Float)
    city_id = Column(Integer, ForeignKey('cities_weather.city_id'))
    city = relationship('CitiesWeather', back_populates='hotels')
    score = Column(Float)
    rating = Column(Float)
    
Base.metadata.create_all(engine)

2022-11-09 13:33:06,715 INFO sqlalchemy.engine.Engine BEGIN (implicit)


2022-11-09 13:33:06 [sqlalchemy.engine.Engine] INFO: BEGIN (implicit)


2022-11-09 13:33:06,721 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-09 13:33:06 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-09 13:33:06,732 INFO sqlalchemy.engine.Engine [cached since 1443s ago] {'name': 'cities_weather'}


...

2022-11-09 13:33:06 [sqlalchemy.engine.Engine] INFO: 
CREATE TABLE hotels (
	hotel_id SERIAL NOT NULL, 
	name VARCHAR, 
	url VARCHAR, 
	description VARCHAR, 
	address VARCHAR, 
	lat FLOAT, 
	lon FLOAT, 
	city_id INTEGER, 
	score FLOAT, 
	rating FLOAT, 
	PRIMARY KEY (hotel_id), 
	FOREIGN KEY(city_id) REFERENCES cities_weather (city_id)
)




2022-11-09 13:33:06,810 INFO sqlalchemy.engine.Engine [no key 0.00673s] {}


2022-11-09 13:33:06 [sqlalchemy.engine.Engine] INFO: [no key 0.00673s] {}


2022-11-09 13:33:06,841 INFO sqlalchemy.engine.Engine COMMIT


2022-11-09 13:33:06 [sqlalchemy.engine.Engine] INFO: COMMIT


In [82]:
# Add the data to their respective table
cities_weather.to_sql(con=engine, name=CitiesWeather.__tablename__, if_exists='append', index=False)
hotels.to_sql(con=engine, name=Hotels.__tablename__, if_exists='append', index=False)

2022-11-09 13:33:12,825 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-09 13:33:12 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-09 13:33:12,837 INFO sqlalchemy.engine.Engine [cached since 1449s ago] {'name': 'cities_weather'}


2022-11-09 13:33:12 [sqlalchemy.engine.Engine] INFO: [cached since 1449s ago] {'name': 'cities_weather'}


2022-11-09 13:33:12,893 INFO sqlalchemy.engine.Engine BEGIN (implicit)


...

2022-11-09 13:33:12 [sqlalchemy.engine.Engine] INFO: INSERT INTO cities_weather (city_id, city, lat, lon, mean_feels_like, mean_pop) VALUES (%(city_id)s, %(city)s, %(lat)s, %(lon)s, %(mean_feels_like)s, %(mean_pop)s)


2022-11-09 13:33:12,914 INFO sqlalchemy.engine.Engine [generated in 0.01166s] ({'city_id': 0, 'city': 'Cassis', 'lat': 43.2140359, 'lon': 5.5396318, 'mean_feels_like': 19, 'mean_pop': 0.0}, {'city_id': 1, 'city': 'Collioure', 'lat': 42.52505, 'lon': 3.0831554, 'mean_feels_like': 19, 'mean_pop': 0.0}, {'city_id': 2, 'city': 'Carcassonne', 'lat': 43.2130358, 'lon': 2.3491069, 'mean_feels_like': 19, 'mean_pop': 0.0}, {'city_id': 3, 'city': 'Marseille', 'lat': 43.2961743, 'lon': 5.3699525, 'mean_feels_like': 18, 'mean_pop': 0.0}, {'city_id': 4, 'city': 'Aix en Provence', 'lat': 43.5298424, 'lon': 5.4474738, 'mean_feels_like': 18, 'mean_pop': 0.0})


2022-11-09 13:33:12 [sqlalchemy.engine.Engine] INFO: [generated in 0.01166s] ({'city_id': 0, 'city': 'Cassis', 'lat': 43.2140359, 'lon': 5.5396318, 'mean_feels_like': 19, 'mean_pop': 0.0}, {'city_id': 1, 'city': 'Collioure', 'lat': 42.52505, 'lon': 3.0831554, 'mean_feels_like': 19, 'mean_pop': 0.0}, {'city_id': 2, 'city': 'Carcassonne', 'lat': 43.2130358, 'lon': 2.3491069, 'mean_feels_like': 19, 'mean_pop': 0.0}, {'city_id': 3, 'city': 'Marseille', 'lat': 43.2961743, 'lon': 5.3699525, 'mean_feels_like': 18, 'mean_pop': 0.0}, {'city_id': 4, 'city': 'Aix en Provence', 'lat': 43.5298424, 'lon': 5.4474738, 'mean_feels_like': 18, 'mean_pop': 0.0})


2022-11-09 13:33:12,947 INFO sqlalchemy.engine.Engine COMMIT


2022-11-09 13:33:12 [sqlalchemy.engine.Engine] INFO: COMMIT


In [86]:
# Verify that everything is working fine !
query = text('SELECT * FROM cities_weather c INNER JOIN hotels h ON c.city_id=h.city_id')
pd.read_sql(query,engine)

2022-11-09 13:37:01,186 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-09 13:37:01 [sqlalchemy.engine.Engine] INFO: select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s


2022-11-09 13:37:01,202 INFO sqlalchemy.engine.Engine [cached since 1678s ago] {'name': 'SELECT * FROM cities_weather c INNER JOIN hotels h ON c.city_id=h.city_id'}


2022-11-09 13:37:01 [sqlalchemy.engine.Engine] INFO: [cached since 1678s ago] {'name': 'SELECT * FROM cities_weather c INNER JOIN hotels h ON c.city_id=h.city_id'}


2022-11-09 13:37:01,264 INFO sqlalchemy.engine.Engine SELECT * FROM cities_weather c INNER JOIN hotels h ON c.city_id=h.city_id


2022-11-09 13:37:01 [sqlalchemy.engine.Engine] INFO: SELECT * FROM cities_weather c INNER JOIN hotels h ON c.city_id=h.city_id


2022-11-09 13:37:01,269 INFO sqlalchemy.engine.Engine [generated in 0.00527s] {}


2022-11-09 13:37:01 [sqlalchemy.engine.Engine] INFO: [generated in 0.00527s] {}


,city_id,city,lat,lon,mean_feels_like,mean_pop,hotel_id,name,url,description,address,lat,lon,city_id,score,rating
0,0,Cassis,43.214036,5.539632,19,0.0,0,Home Cassis - Maison Julou - Piscine chauffée,https://www.booking.com/hotel/fr/home-cassis-m...,"Offrant une vue sur le jardin, le Home Cassis ...","23 Avenue de la Marne, 13260 Cassis, France",43.222080,5.537927,0,7.6,3.0
1,0,Cassis,43.214036,5.539632,19,0.0,1,LA FALAISE PARADIS Vue mer 180°,https://www.booking.com/hotel/fr/la-falaise-pa...,Vous pouvez bénéficier d'une réduction Genius ...,"3 Avenue de Provence, 13260 Cassis, France",43.215295,5.546231,0,7.5,3.0
2,0,Cassis,43.214036,5.539632,19,0.0,2,Le Cleli,https://www.booking.com/hotel/fr/le-cleli.fr.html,Vous pouvez bénéficier d'une réduction Genius ...,"11 Rue de la Ciotat, 13260 Cassis, France",43.215098,5.540487,0,8.1,3.0
3,0,Cassis,43.214036,5.539632,19,0.0,3,appartement cosy à 10 mètres du port de Cassis,https://www.booking.com/hotel/fr/appartement-c...,Vous pouvez bénéficier d'une réduction Genius ...,"6 Rue Bremond, 13260 Cassis, France",43.215335,5.536861,0,8.3,3.0
4,0,Cassis,43.214036,5.539632,19,0.0,4,"Cassis, le Grand Bleu, triplex vue mer, port p...",https://www.booking.com/hotel/fr/le-grand-bleu...,Vous pouvez bénéficier d'une réduction Genius ...,"3 Avenue Auguste Favier, 13260 Cassis, France",43.220804,5.542296,0,9.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,Cassis,43.214036,5.539632,19,0.0,95,Le Duplex,https://www.booking.com/hotel/fr/le-duplex-cas...,Vous pouvez bénéficier d'une réduction Genius ...,"11 Rue de la Ciotat, 13260 Cassis, France",43.215055,5.540573,0,8.2,3.0
96,0,Cassis,43.214036,5.539632,19,0.0,96,Cassis Ma Vie Là,https://www.booking.com/hotel/fr/cassis-ma-vie...,Vous pouvez bénéficier d'une réduction Genius ...,"10 ter avenue Augustin Isnard, 13260 Cassis, F...",43.216905,5.541446,0,9.1,3.0
97,0,Cassis,43.214036,5.539632,19,0.0,97,appartement du pêcheur à Cassis à 10 mètres du...,https://www.booking.com/hotel/fr/appartement-d...,Vous pouvez bénéficier d'une réduction Genius ...,"6 Rue Bremond, 13260 Cassis, France",43.215340,5.536858,0,8.6,3.0
98,0,Cassis,43.214036,5.539632,19,0.0,98,BALCON SUR MER vue mer,https://www.booking.com/hotel/fr/balcon-sur-me...,Vous pouvez bénéficier d'une réduction Genius ...,Résidence La Rade Bt C6 av Amiral Ganteaume 13...,43.215894,5.534763,0,8.0,4.0
